In [2]:
#imports and setup
import numpy as np
import cv2 as cv
import os

In [3]:
'''loading video names into a list for easier access with absolute path,
video names can be access by calling the index of the video_lst and will contain 
the current absolute path, asumes that the video folder is in the same folder as this file'''

video_path = "videos"
full_path = os.path.abspath(video_path)

video_lst = []
v_dir = os.listdir(full_path)
for name in v_dir:
    video_lst.append(full_path+'/'+name)
print(video_lst[0]) 

/Users/daryashitova/Documents/MA_AI/WS22:23/Computer Vision/project/computer_vision_prototype-main/videos/video_026.mp4


In [3]:
#loading video into memory
cap = cv.VideoCapture(video_lst[0])

In [ ]:
'''Playing one video file'''

while cap.isOpened():
    ret, frame = cap.read()
    # if frame is read correctly ret is True
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break
    
    '''cv_waitkey() defines miliseconds between frames, higher values play the video slower
    ord('q') defines a key to exit the video'''
    cv.imshow('frame',frame)
    if cv.waitKey(25) & 0xFF == ord('q'):
        break
cap.release() #after release from memory the video needs to be recaptured
cv.destroyAllWindows()

In [ ]:
'''Testing background substraction with morphological noise reduction
    opencv has KNN and MOG2 BGS build in'''

cap = cv.VideoCapture(video_lst[0])
backSub = cv.createBackgroundSubtractorKNN()
#backSub = cv.createBackgroundSubtractorMOG2()

#kernel for morphological noise reduction
#cv.MORPH_ELLIPSE
kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE, (3, 3));
while cap.isOpened():
    ret, frame = cap.read()
    if frame is None:
        break
    
    fgMask = backSub.apply(frame)
    fgMask = cv.morphologyEx(fgMask, cv.MORPH_OPEN, kernel);

    #cv.imshow('Frame', frame)
    cv.imshow('FG Mask', fgMask)
    
    if cv.waitKey(25) & 0xFF == ord('q'):
        break
    
cap.release()
cv.destroyAllWindows()

In [4]:
# Daria - Euclidean Distance Tracker

import math


class EuclideanDistTracker:
    def __init__(self):
        # Store the center positions of the objects
        self.center_points = {}
        # Keep the count of the IDs
        # each time a new object id detected, the count will increase by one
        self.id_count = 0


    def update(self, objects_rect):
        # Objects boxes and ids
        objects_bbs_ids = []

        # Get center point of new object
        for rect in objects_rect:
            x, y, w, h = rect
            cx = (x + x + w) // 2
            cy = (y + y + h) // 2

            # Find out if that object was detected already
            same_object_detected = False
            for id, pt in self.center_points.items():
                dist = math.hypot(cx - pt[0], cy - pt[1])

                if dist < 25:
                    self.center_points[id] = (cx, cy)
                    print(self.center_points)
                    objects_bbs_ids.append([x, y, w, h, id])
                    same_object_detected = True
                    break

            # New object is detected we assign the ID to that object
            if same_object_detected is False:
                self.center_points[self.id_count] = (cx, cy)
                objects_bbs_ids.append([x, y, w, h, self.id_count])
                self.id_count += 1

        # Clean the dictionary by center points to remove IDS not used anymore
        new_center_points = {}
        for obj_bb_id in objects_bbs_ids:
            _, _, _, _, object_id = obj_bb_id
            center = self.center_points[object_id]
            new_center_points[object_id] = center

        # Update dictionary with IDs not used removed
        self.center_points = new_center_points.copy()
        return objects_bbs_ids


In [4]:
# import the necessary packages
from scipy.spatial import distance as dist
from collections import OrderedDict
import numpy as np
class CentroidTracker():
	def __init__(self, maxDisappeared=50):
		# initialize the next unique object ID along with two ordered
		# dictionaries used to keep track of mapping a given object
		# ID to its centroid and number of consecutive frames it has
		# been marked as "disappeared", respectively
		self.nextObjectID = 0
		self.objects = OrderedDict()
		self.disappeared = OrderedDict()
		# store the number of maximum consecutive frames a given
		# object is allowed to be marked as "disappeared" until we
		# need to deregister the object from tracking
		self.maxDisappeared = maxDisappeared

	def register(self, centroid):
		# when registering an object we use the next available object
		# ID to store the centroid
		self.objects[self.nextObjectID] = centroid
		self.disappeared[self.nextObjectID] = 0
		self.nextObjectID += 1

	def deregister(self, objectID):
		# to deregister an object ID we delete the object ID from
		# both of our respective dictionaries
		del self.objects[objectID]
		del self.disappeared[objectID]

	def update(self, rects):
		# check to see if the list of input bounding box rectangles
		# is empty
		if len(rects) == 0:
			# loop over any existing tracked objects and mark them
			# as disappeared
			for objectID in list(self.disappeared.keys()):
				self.disappeared[objectID] += 1
				# if we have reached a maximum number of consecutive
				# frames where a given object has been marked as
				# missing, deregister it
				if self.disappeared[objectID] > self.maxDisappeared:
					self.deregister(objectID)
			# return early as there are no centroids or tracking info
			# to update
			return self.objects
		# initialize an array of input centroids for the current frame
		inputCentroids = np.zeros((len(rects), 2), dtype="int")
		# loop over the bounding box rectangles
		for (i, (startX, startY, endX, endY)) in enumerate(rects):
			# use the bounding box coordinates to derive the centroid
			cX = (startX + startX + endX) // 2.0
			cY = (startY + startY + endY) // 2.0
			inputCentroids[i] = (cX, cY)
		# if we are currently not tracking any objects take the input
		# centroids and register each of them
		if len(self.objects) == 0:
			for i in range(0, len(inputCentroids)):
				self.register(inputCentroids[i])
		# otherwise, are are currently tracking objects so we need to
		# try to match the input centroids to existing object
		# centroids
		else:
			# grab the set of object IDs and corresponding centroids
			objectIDs = list(self.objects.keys())
			objectCentroids = list(self.objects.values())
			# compute the distance between each pair of object
			# centroids and input centroids, respectively -- our
			# goal will be to match an input centroid to an existing
			# object centroid
			D = dist.cdist(np.array(objectCentroids), inputCentroids)
			# in order to perform this matching we must (1) find the
			# smallest value in each row and then (2) sort the row
			# indexes based on their minimum values so that the row
			# with the smallest value is at the *front* of the index
			# list
			rows = D.min(axis=1).argsort()
			# next, we perform a similar process on the columns by
			# finding the smallest value in each column and then
			# sorting using the previously computed row index list
			cols = D.argmin(axis=1)[rows]
			# in order to determine if we need to update, register,
			# or deregister an object we need to keep track of which
			# of the rows and column indexes we have already examined
			usedRows = set()
			usedCols = set()
			# loop over the combination of the (row, column) index
			# tuples
			for (row, col) in zip(rows, cols):
				# if we have already examined either the row or
				# column value before, ignore it
				# val
				if row in usedRows or col in usedCols:
					continue
				# otherwise, grab the object ID for the current row,
				# set its new centroid, and reset the disappeared
				# counter
				objectID = objectIDs[row]
				self.objects[objectID] = inputCentroids[col]
				self.disappeared[objectID] = 0
				# indicate that we have examined each of the row and
				# column indexes, respectively
				usedRows.add(row)
				usedCols.add(col)
			# compute both the row and column index we have NOT yet
			# examined
			unusedRows = set(range(0, D.shape[0])).difference(usedRows)
			unusedCols = set(range(0, D.shape[1])).difference(usedCols)
			# in the event that the number of object centroids is
			# equal or greater than the number of input centroids
			# we need to check and see if some of these objects have
			# potentially disappeared
			if D.shape[0] >= D.shape[1]:
				# loop over the unused row indexes
				for row in unusedRows:
					# grab the object ID for the corresponding row
					# index and increment the disappeared counter
					objectID = objectIDs[row]
					self.disappeared[objectID] += 1
					# check to see if the number of consecutive
					# frames the object has been marked "disappeared"
					# for warrants deregistering the object
					if self.disappeared[objectID] > self.maxDisappeared:
						self.deregister(objectID)
			# otherwise, if the number of input centroids is greater
			# than the number of existing object centroids we need to
			# register each new input centroid as a trackable object
			else:
				for col in unusedCols:
					self.register(inputCentroids[col])
		# return the set of trackable objects
		return self.objects

/Users/daryashitova/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:
# tracker types in opencv -> needs different version of opencv to run
tracker_types = ['BOOSTING', 'MIL','KCF', 'TLD', 'MEDIANFLOW', 'GOTURN', 'MOSSE', 'CSRT']


tracker = tracker_types[1]

if tracker == 'BOOSTING':
    tracker = cv.legacy.TrackerBoosting_create()
if tracker == 'MIL':
    tracker = cv.TrackerMIL_create() 
if tracker == 'KCF':
    tracker = cv.TrackerKCF_create() 
if tracker == 'TLD':
    tracker = cv.legacy.TrackerTLD_create() 
if tracker == 'MEDIANFLOW':
    tracker = cv.legacy.TrackerMedianFlow_create() 
# if tracker_type == 'GOTURN':
#     tracker = cv.TrackerGOTURN_create()
if tracker == 'MOSSE':
    tracker = cv.legacy.TrackerMOSSE_create()
if tracker == "CSRT":
    tracker = cv.TrackerCSRT_create()

print(f"Current tracker type: {tracker}")

Current tracker type: < cv2.TrackerMIL 0x7fa061d87290>


In [6]:
# Utility Functions
# Compute the frame difference
def frame_diff(prev_frame, cur_frame, next_frame):
    diff_frames1 = cv.absdiff(next_frame, cur_frame)
    # Absolute difference between current frame and previous frame
    diff_frames2 = cv.absdiff(cur_frame, prev_frame)
    # Return the result of bitwise 'AND' between the above two resultant images
    #gives better result than simple substraction
    return cv.bitwise_and(diff_frames1, diff_frames2)

def get_frame(cap):
    ret, frame = cap.read()
    # Resize the image not needed
    '''frame = cv.resize(frame, None, fx=scaling_factor,
        fy=scaling_factor, interpolation=cv.INTER_AREA)'''
    return frame

#finds and draws contours over pixels if greater than threshold
def construct_contours(frame, contour_threshold):
    frame = cv.dilate(frame, (1,1), iterations=1)
    contours, hierarchy = cv.findContours(frame, cv.RETR_TREE, 
                                           cv.CHAIN_APPROX_SIMPLE)
    '''cv.drawContours(frame_th, contours=contours, contourIdx=-1, 
                     color=(0, 255, 0), thickness=2, lineType=cv.LINE_AA)'''
    detections=[]
    for contour in contours:
      if cv.contourArea(contour) < contour_threshold:
        continue
      (x, y, w, h) = cv.boundingRect(contour)
      #cv.rectangle(frame, pt1=(x, y), pt2=(x + w, y + h), color=(0, 255, 0), thickness=2)
      detections.append([x,y,w,h])
    
    return detections

    #applying blur will possibly extend with other option for preperation
def prepare_frames(frames,kernel):
    result = []
    for frame in frames:
        result.append(cv.GaussianBlur(frame, kernel, 0))
    return result[0], result[1], result[2]

In [7]:
# Trying out tracker:


'''Testing Frame differencing code based on:
https://github.com/infoaryan/OPENCV-PYTHON-Zero-to-One-Course-Resources/blob/master/Video%2031%20-%20Frame%20Differencing/frame_differencing.py
Expanded with: blur, dilation and morphological noise reduction and contour finding'''

#video to capture

video_lst = sorted(video_lst)
video = video_lst[10]
print(video)

kernel = np.ones((7,7)) #kerne for dilation
kernel_blur = (3,3)
kernel_morph = cv.getStructuringElement(cv.MORPH_ELLIPSE, (7, 7)) #kernel for morphology
THRESHOLD = 20 #when to detect difference
VALUE = 255 #which value to assign to difference

#threshold to calculate
threshold_method = cv.THRESH_BINARY
tracker = EuclideanDistTracker()


cap = cv.VideoCapture(video)
prev_frame = get_frame(cap)
cur_frame = get_frame(cap)
#next_frame = get_frame(cap)
ret, next_frame = cap.read()

print(type(next_frame))

centerpoints = []

trackerrInit = False


# Iterating over all frames and applying difference and dilation
while True:
    frames = [prev_frame,cur_frame,next_frame]
    #applying preprocessing
    if type(next_frame) is None:
         break
    prev_frame, cur_frame, next_frame = prepare_frames(frames, kernel_blur)
    
    #calculating difference
    frame_difference = frame_diff(prev_frame, cur_frame, next_frame)
    #cv.imshow('frame diff', frame_difference)
    #applying dilation
    frame_difference = cv.dilate(frame_difference, kernel)
    #applying morphological noise reduction
    frame_difference = cv.morphologyEx(frame_difference, cv.MORPH_OPEN, kernel_morph)
    #converting into greyscale for contour finding
    frame_difference = cv.cvtColor(frame_difference, cv.COLOR_BGR2GRAY)
    #applying thresholds
    ret, frame_th = cv.threshold(frame_difference, THRESHOLD, VALUE, threshold_method)
    #finding and drawing contours on given frame with given threshold
    detections = construct_contours(frame_th, 30)
    # detections -> bb coordinates
    
    #cv.imshow("Difference", frame_difference)
    cv.imshow("After Threshold", frame_th)
    #cv.imshow("Frame", cur_frame)

    # if not trackerrInit:
    #     # Initialize tracker with first frame and bounding box
    #     #tracker.init(cur_frame,)
    #     trackerrInit = True


    boxes_ids = tracker.update(detections)

    #boxes_ids = tracker.update(cur_frame)


    # detection
    for box_id in boxes_ids:
        
        x,y,w,h,id = box_id

        
        # get coordinates of center point
        cx = int((x + x + w) / 2)  
        cy = int((y + y + h) / 2)
        
        centerpoints.append((cx, cy))


        cv.putText(cur_frame, "frame:" + str(cap.get(cv.CAP_PROP_POS_FRAMES)), (15, 15),
               cv.FONT_HERSHEY_SIMPLEX, 0.5 , (0, 218, 0))
        cv.putText(cur_frame,  str(id),(x,y-15), cv.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,255), 2)
        cv.rectangle(cur_frame, (x,y),(x+w, y+h), (0,255,0), 2)
        
#        for point in centerpoints:
 #           cv.circle(cur_frame, point, 3, (0,0,255), -1 )
        #cv.imshow('frame', cur_frame)

    # show circle around centerpoints
    for point in centerpoints:
        cv.circle(cur_frame, point, 3, (0,0,255), 0 )
    cv.imshow('frame', cur_frame)


    # Update the variables
    prev_frame = cur_frame
    cur_frame = next_frame
    # next_frame = get_frame(cap)
    ret, next_frame = cap.read()
    
    if not ret:
         break

    if cv.waitKey(25) & 0xFF == ord('q'):
        break    
cap.release()
cv.waitKey(1)

cv.destroyAllWindows()
for i in range (1,5):
    cv.waitKey(1)
#cap.release()



/Users/daryashitova/Documents/MA_AI/WS22:23/Computer Vision/project/computer_vision_prototype-main/videos/video_005.mp4
<class 'numpy.ndarray'>
{0: (129, 140)}
{0: (128, 138)}
{0: (129, 139)}
{0: (130, 139)}
{0: (132, 140)}
{0: (129, 140)}
{0: (132, 139)}
{0: (134, 138)}
{0: (132, 138)}
{0: (131, 137)}
{0: (132, 137)}
{0: (129, 139)}
{0: (131, 138)}
{0: (129, 139)}
{0: (130, 139)}
{0: (128, 138)}
{0: (129, 137)}
{0: (129, 140)}
{0: (129, 140)}
{0: (129, 139)}
{0: (129, 139)}
{0: (129, 140)}
{0: (129, 140)}
{0: (129, 140), 1: (225, 272)}
{1: (225, 271), 0: (129, 140)}
{1: (225, 271), 0: (129, 139)}
{1: (225, 270), 0: (129, 139)}
{1: (225, 270), 0: (130, 139)}
{1: (225, 271), 0: (130, 139)}
{1: (225, 271), 0: (129, 139)}
{1: (225, 271), 0: (129, 139)}
{1: (225, 271), 0: (130, 138)}
{1: (225, 271), 0: (130, 138)}
{1: (225, 271), 0: (129, 139)}
{1: (225, 271), 0: (129, 139)}
{1: (225, 271), 0: (129, 138)}
{1: (225, 271), 0: (129, 138)}
{1: (225, 271), 0: (132, 139)}
{1: (225, 271), 0: (132

In [20]:
#With the option to save videos
# Trying out tracker:
'''Testing Frame differencing code based on:
https://github.com/infoaryan/OPENCV-PYTHON-Zero-to-One-Course-Resources/blob/master/Video%2031%20-%20Frame%20Differencing/frame_differencing.py
Expanded with: blur, dilation and morphological noise reduction and contour finding'''

#video to capture

video_lst = sorted(video_lst)
video = video_lst[10]
print(video)

kernel = np.ones((7,7)) #kerne for dilation
kernel_blur = (3,3)
kernel_morph = cv.getStructuringElement(cv.MORPH_ELLIPSE, (7, 7)) #kernel for morphology
THRESHOLD = 20 #when to detect difference
VALUE = 255 #which value to assign to difference

#threshold to calculate
threshold_method = cv.THRESH_BINARY
tracker = EuclideanDistTracker()



cap = cv.VideoCapture(video)
prev_frame = get_frame(cap)
cur_frame = get_frame(cap)
#next_frame = get_frame(cap)
ret, next_frame = cap.read()

print(type(next_frame))

centerpoints = []

trackerrInit = False

'''Check if directory exists, if not, create it'''
import os

# You should change 'test' to your preferred folder.
dir = ("outputs")
check = os.path.isdir(dir)

# If folder doesn't exist, then create it.
if not check:
    os.makedirs(dir)
    print("created folder : ", dir)

else:
    print(dir, "folder already exists.")

file_name = video.split("/")[-1].split(".")
file_name[0] = file_name[0]+"_out"
output_name = ".".join(file_name)
full_output_path = f"{dir}/{output_name}"

result = cv.VideoWriter(full_output_path, cv.VideoWriter_fourcc(*'XVID'), 20, (640, 512))


# Iterating over all frames and applying difference and dilation
while True:
    frames = [prev_frame,cur_frame,next_frame]
    #applying preprocessing
    if type(next_frame) is None:
         break
    prev_frame, cur_frame, next_frame = prepare_frames(frames, kernel_blur)
    
    #calculating difference
    frame_difference = frame_diff(prev_frame, cur_frame, next_frame)
    #cv.imshow('frame diff', frame_difference)
    #applying dilation
    frame_difference = cv.dilate(frame_difference, kernel)
    #applying morphological noise reduction
    frame_difference = cv.morphologyEx(frame_difference, cv.MORPH_OPEN, kernel_morph)
    #converting into greyscale for contour finding
    frame_difference = cv.cvtColor(frame_difference, cv.COLOR_BGR2GRAY)
    #applying thresholds
    ret, frame_th = cv.threshold(frame_difference, THRESHOLD, VALUE, threshold_method)
    #finding and drawing contours on given frame with given threshold
    detections = construct_contours(frame_th, 30)
    # detections -> bb coordinates
    
    #cv.imshow("Difference", frame_difference)
    cv.imshow("After Threshold", frame_th)
    #cv.imshow("Frame", cur_frame)

    # if not trackerrInit:
    #     # Initialize tracker with first frame and bounding box
    #     #tracker.init(cur_frame,)
    #     trackerrInit = True


    boxes_ids = tracker.update(detections)

    #boxes_ids = tracker.update(cur_frame)


    # detection
    for box_id in boxes_ids:
        
        x,y,w,h,id = box_id

        
        # get coordinates of center point
        cx = int((x + x + w) / 2)  
        cy = int((y + y + h) / 2)
        
        centerpoints.append((cx, cy))


        cv.putText(cur_frame, "frame:" + str(cap.get(cv.CAP_PROP_POS_FRAMES)), (15, 15),
               cv.FONT_HERSHEY_SIMPLEX, 0.5 , (0, 218, 0))
        cv.putText(cur_frame,  str(id),(x,y-15), cv.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,255), 2)
        cv.rectangle(cur_frame, (x,y),(x+w, y+h), (0,255,0), 2)
        
#        for point in centerpoints:
 #           cv.circle(cur_frame, point, 3, (0,0,255), -1 )
        #cv.imshow('frame', cur_frame)

    # show circle around centerpoints
    for point in centerpoints:
        cv.circle(cur_frame, point, 1, (0,255,0), 0 )
    cv.imshow('frame', cur_frame)

    result.write(cur_frame)
    # Update the variables
    prev_frame = cur_frame
    cur_frame = next_frame
    # next_frame = get_frame(cap)
    ret, next_frame = cap.read()
    
    if not ret:
         break

    if cv.waitKey(25) & 0xFF == ord('q'):
        break    
cap.release()
result.release()
cv.waitKey(1)

cv.destroyAllWindows()
for i in range (1,5):
    cv.waitKey(1)
#cap.release()



/Users/daryashitova/Documents/MA_AI/WS22:23/Computer Vision/project/computer_vision_prototype-main/videos/video_010.mp4
<class 'numpy.ndarray'>
outputs folder already exists.


OpenCV: FFMPEG: tag 0x44495658/'XVID' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


432 359
419 355
{1: (419, 355)}
419 355
{1: (424, 363)}
{1: (417, 354)}
424 363
417 354
{1: (419, 356)}
419 356
423 363
{2: (422, 360)}
422 360
{2: (421, 358)}
421 358
{2: (423, 363)}
{2: (430, 358)}
423 363
430 358
{2: (425, 358)}
425 358
{2: (421, 360)}
421 360
{2: (423, 358)}
423 358
{2: (422, 358)}
422 358
{2: (420, 359)}
420 359
420 359
{3: (422, 357)}
422 357
{3: (418, 359)}
{3: (428, 355)}
418 359
428 355
{3: (419, 358)}
419 358
{3: (420, 356)}
420 356
417 358
{4: (419, 355)}
419 355
{4: (418, 356)}
418 356
{4: (426, 356)}
{4: (417, 359)}
426 356
417 359
418 356
{5: (416, 356)}
416 356
413 350
{6: (413, 352)}
413 352
415 357
{7: (417, 355)}
417 355
{7: (407, 347)}
407 347
{7: (416, 353)}
416 353
{7: (407, 347)}
407 347
419 356
{8: (417, 356)}
417 356
{8: (416, 355)}
416 355
{8: (418, 359)}
{8: (406, 347)}
418 359
406 347
419 359
{9: (415, 356)}
415 356
{9: (413, 353)}
413 353
{9: (413, 353)}
413 353
{9: (416, 359)}
{9: (410, 351)}
416 359
410 351
{9: (407, 349)}
407 349
{9: (413

In [8]:
#With the option to save videos
# Trying out Centroid tracker:
'''Testing Frame differencing code based on:
https://github.com/infoaryan/OPENCV-PYTHON-Zero-to-One-Course-Resources/blob/master/Video%2031%20-%20Frame%20Differencing/frame_differencing.py
Expanded with: blur, dilation and morphological noise reduction and contour finding'''

#video to capture

video_lst = sorted(video_lst)
video = video_lst[10]
print(video)

kernel = np.ones((7,7)) #kerne for dilation
kernel_blur = (3,3)
kernel_morph = cv.getStructuringElement(cv.MORPH_ELLIPSE, (7, 7)) #kernel for morphology
THRESHOLD = 20 #when to detect difference
VALUE = 255 #which value to assign to difference

#threshold to calculate
threshold_method = cv.THRESH_BINARY
tracker = CentroidTracker()


cap = cv.VideoCapture(video)
prev_frame = get_frame(cap)
cur_frame = get_frame(cap)
#next_frame = get_frame(cap)
ret, next_frame = cap.read()

print(type(next_frame))

centerpoints = []

trackerrInit = False

'''Check if directory exists, if not, create it'''
import os

# You should change 'test' to your preferred folder.
dir = ("outputs")
check = os.path.isdir(dir)

# If folder doesn't exist, then create it.
if not check:
    os.makedirs(dir)
    print("created folder : ", dir)

else:
    print(dir, "folder already exists.")

file_name = video.split("/")[-1].split(".")
file_name[0] = file_name[0]+"_out"
output_name = ".".join(file_name)
full_output_path = f"{dir}/{output_name}"

result = cv.VideoWriter(full_output_path, cv.VideoWriter_fourcc(*'XVID'), 20, (640, 512))


# Iterating over all frames and applying difference and dilation
while True:
    frames = [prev_frame,cur_frame,next_frame]
    #applying preprocessing
    if type(next_frame) is None:
         break
    prev_frame, cur_frame, next_frame = prepare_frames(frames, kernel_blur)
    
    #calculating difference
    frame_difference = frame_diff(prev_frame, cur_frame, next_frame)
    #cv.imshow('frame diff', frame_difference)
    #applying dilation
    frame_difference = cv.dilate(frame_difference, kernel)
    #applying morphological noise reduction
    frame_difference = cv.morphologyEx(frame_difference, cv.MORPH_OPEN, kernel_morph)
    #converting into greyscale for contour finding
    frame_difference = cv.cvtColor(frame_difference, cv.COLOR_BGR2GRAY)
    #applying thresholds
    ret, frame_th = cv.threshold(frame_difference, THRESHOLD, VALUE, threshold_method)
    #finding and drawing contours on given frame with given threshold
    detections = construct_contours(frame_th, 30)
    # detections -> bb coordinates
    
    #cv.imshow("Difference", frame_difference)
    cv.imshow("After Threshold", frame_th)
    #cv.imshow("Frame", cur_frame)

    # if not trackerrInit:
    #     # Initialize tracker with first frame and bounding box
    #     #tracker.init(cur_frame,)
    #     trackerrInit = True


    boxes_ids = tracker.update(detections)

    
    # detection
    for (objectID, centroid) in boxes_ids.items():
        
        centerpoints.append((centroid[0], centroid[1]))
        text = f"ID {objectID}"

        cv.putText(cur_frame, text, (centroid[0], centroid[1]-15),
               cv.FONT_HERSHEY_SIMPLEX, 0.5 , (0, 255, 0), 2)
        #cv.putText(cur_frame,  str(id),(x,y-15), cv.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,255), 2)
        cv.rectangle(cur_frame, (centroid[0]-10, centroid[1]-10), (centroid[0]+10, centroid[1]+10), (0,255,0), 2)
        
#        for point in centerpoints:
 #           cv.circle(cur_frame, point, 3, (0,0,255), -1 )
        #cv.imshow('frame', cur_frame)

    # show circle around centerpoints
    #for point in centerpoints:
    for i, point in enumerate(centerpoints[1:]):
        cv.line(cur_frame, centerpoints[i], point, (0,255,0), 1) 
    #    cv.circle(cur_frame, point, 1, (0,255,0), 0 )
    cv.imshow('frame', cur_frame)

    result.write(cur_frame)
    # Update the variables
    prev_frame = cur_frame
    cur_frame = next_frame
    # next_frame = get_frame(cap)
    ret, next_frame = cap.read()
    
    if not ret:
         break

    if cv.waitKey(25) & 0xFF == ord('q'):
        break    
cap.release()
result.release()
cv.waitKey(1)

cv.destroyAllWindows()
for i in range (1,5):
    cv.waitKey(1)
#cap.release()



/Users/daryashitova/Documents/MA_AI/WS22:23/Computer Vision/project/computer_vision_prototype-main/videos/video_010.mp4
<class 'numpy.ndarray'>
outputs folder already exists.


OpenCV: FFMPEG: tag 0x44495658/'XVID' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
